In [1]:
!pip install ultralytics -q
!pip install roboflow -q


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from ultralytics.nn.modules import (
    AIFI,
    C1,
    C2,
    C2PSA,
    C3,
    C3TR,
    ELAN1,
    OBB,
    PSA,
    SPP,
    SPPELAN,
    SPPF,
    A2C2f,
    AConv,
    ADown,
    Bottleneck,
    BottleneckCSP,
    C2f,
    C2fAttn,
    C2fCIB,
    C2fPSA,
    C3Ghost,
    C3k2,
    C3x,
    CBFuse,
    CBLinear,
    Classify,
    Concat,
    Conv,
    Conv2,
    ConvTranspose,
    Detect,
    DWConv,
    DWConvTranspose2d,
    Focus,
    GhostBottleneck,
    GhostConv,
    HGBlock,
    HGStem,
    ImagePoolingAttn,
    Index,
    LRPCHead,
    Pose,
    RepC3,
    RepConv,
    RepNCSPELAN4,
    RepVGGDW,
    ResNetLayer,
    RTDETRDecoder,
    SCDown,
    Segment,
    TorchVision,
    WorldDetect,
    YOLOEDetect,
    YOLOESegment,
    v10Detect,
)

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('../../myconfig.config')
print(config['roboflow']['roboflow_api_key'])


['../../myconfig.config']

In [6]:
from roboflow import Roboflow
rf = Roboflow(api_key='u8Ur5ywKLswqwt0uMTXl')
project = rf.workspace("poacher-ifyuf").project("poacher")
version = project.version(2)
dataset = version.download("yolov11", location="../../Dataset/Test_dataset")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ../../Dataset/Test_dataset in yolov11:: 100%|██████████| 1396/1396 [00:01<00:00, 928.36it/s] 


In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/Colab_Notebooks/YOLO_architecture/utillity')

In [2]:
from ultralytics import nn
from ultralytics.nn import modules, tasks
import ultralytics.nn.autobackend as autobackend
from model_classes import Upsample, C2f, SPPF, DFL, Head, Bottleneck, Concat
from model_classes_v11 import C3k2, C2PSA, Conv, Attention, PSABlock, C3k, Bottleneck, C2f
from ultralytics.nn.tasks import parse_model, yaml_model_load
from copy import deepcopy


def patch_ultralytics_layers(custom_classes):
    submodules = [
        nn,                          # ultralytics.nn
        modules,                     # ultralytics.nn.modules
        tasks,                       # ultralytics.nn.tasks
        autobackend,                 # ultralytics.nn.autobackend
    ]

    for cls in custom_classes:
        for mod in submodules:
            mod.__dict__[cls.__name__] = cls

def build_model(model_yaml, version='n', ch=3, verbose=False, custom_classes=[]):

  patch_ultralytics_layers(custom_classes)

  mod_ya = model_yaml if isinstance(model_yaml, dict) else yaml_model_load(model_yaml)
  mod_ya['scale'] = version.lower()
  model, save  = parse_model(deepcopy(mod_ya), ch=ch, verbose=verbose)

  return model,save


In [4]:
from ultralytics import YOLO
m_class = [Upsample, Conv, C2f, SPPF, DFL, Head, Bottleneck, Concat, C3k2, C2PSA, Attention, PSABlock, C3k, Bottleneck, C2f]
model_yaml = "my_yolov11s.yaml"

model_save = build_model(model_yaml, custom_classes=m_class, version='l', verbose=False)

model = YOLO(model_yaml, verbose=True)
print(f"{sum(p.numel() for p in model.parameters()) / 1e6:.2f} million parameters")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  model_classes_v11.Conv                       [3, 32, 3, 2]                 
  1                  -1  1     18560  model_classes_v11.Conv                       [32, 64, 3, 2]                
  2                  -1  1     26080  model_classes_v11.C3k2                       [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  model_classes_v11.Conv                       [128, 128, 3, 2]              
  4                  -1  1    103360  model_classes_v11.C3k2                       [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  model_classes_v11.Conv                       [256, 256, 3, 2]              
  6                  -1  1    346112  model_classes_v11.C3k2                       [256, 256, 1, True]           
  7                  -1  1   1180672  model_classes_v11.Conv                       [256

In [1]:
from ultralytics import YOLO

# Load the YOLOv11l model
model = YOLO("yolo11s.yaml",verbose=True)  # Replace with the path to your model file
print(f"{sum(p.numel() for p in model.parameters()) / 1e6:.2f} million parameters")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [5]:
results = model.train(data=r"C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\data.yaml", epochs=1, imgsz=64)

New https://pypi.org/project/ultralytics/8.3.202 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.173  Python-3.11.3 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.

train: Scanning C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\train\labels... 604 images, 9 backgrounds, 0 corrupt: 100%|██████████| 604/604 [00:02<00:00, 253.88it/s]

train: New cache created: C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\train\labels.cache


val: Fast image access  (ping: 0.40.1 ms, read: 6.01.4 MB/s, size: 56.0 KB)


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\valid\labels... 58 images, 1 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<00:00, 240.87it/s]

...

Plotting labels to C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Ultralytics_work\ultralytics\runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000385, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 64 train, 64 val
Using 0 dataloader workers
Logging results to C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Ultralytics_work\ultralytics\runs\detect\train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      6.586      5.247      4.233         38         64: 100%|██████████| 38/38 [00:26<00:00,  1.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]


                   all         58        122          0          0          0          0

1 epochs completed in 0.009 hours.
Optimizer stripped from C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Ultralytics_work\ultralytics\runs\detect\train\weights\last.pt, 19.1MB
Optimizer stripped from C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Ultralytics_work\ultralytics\runs\detect\train\weights\best.pt, 19.1MB

Validating C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Ultralytics_work\ultralytics\runs\detect\train\weights\best.pt...
Ultralytics 8.3.173  Python-3.11.3 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
my_YOLOv11s summary: 117 layers, 9,423,234 parameters, 0 gradients, 21.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


                   all         58        122          0          0          0          0
                animal          2          2          0          0          0          0
                  bull          1          1          0          0          0          0
                 camel          2          4          0          0          0          0
                  deer          3          3          0          0          0          0
                   dog          2          3          0          0          0          0
              elephant          8          8          0          0          0          0
                   fox          1          1          0          0          0          0
                  lion          2          2          0          0          0          0
               poacher         18         29          0          0          0          0
                ranger         18         32          0          0          0          0
                 rhin